<a href="https://colab.research.google.com/github/MweinbergUmass/Teach_Dimred/blob/main/Collab_Python/tsnedisplay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



In order to understand what Tsne does and how it works we should start by considering the concept of a distance matrix. Consider a matrix X, each of it's rows is a list of numbers that somehow describe the data. For instance, in the first data set we are going to consider, the fisher iris dataset, there are four columns: Sepal length, Sepal width, Petal length, Petal width.


Put another way, someone at some point sat down, grabbed an iris (flower), recorded it's sepal length, width, etc... and now I have them in the format of one flower per row. The reason I'm harping on this is because thinking of rows as going together is deeply important for dimensionality reduction and data science broadly.


Now, back to the distance matrix X:


When I say distance all I mean is, you take datapoint x, and datapoint y, and find the euclidean distance between them.


Note (we're working in two dimensions for now)


In math:


$$d(x,y)=\sqrt{(x_2 -x_1 )^2 +(y_2 -y_1 )^2 }$$

Now in Python:

<pre>
def d(x, y):
    return np.sqrt((x[1] - x[0])**2 + (y[1] - y[0])**2)
</pre>

Now let's get going. We're going to walk through how Tsne does what it does with Python. If you're new to google collab all you have to do should be to hit the run button on the left of each cell and then play with the sliders to see how they update the plots.

In [ ]:
# @title Run this cell, it's the imports.
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, t
from scipy.spatial.distance import pdist, squareform
import ipywidgets as widgets
from IPython.display import display
from matplotlib.animation import FuncAnimation
from matplotlib import cm, colormaps
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
from sklearn.datasets import load_iris





In [ ]:
# @title These are the functions we will be using.
def compute_entropy(current_row, Sigma):
    num = np.exp(-current_row / (2 * Sigma**2))
    num[num == np.max(num)] = 0
    thisP = num / np.sum(num)
    H = -np.sum(thisP * np.log2(thisP + np.finfo(float).eps))
    return H, thisP

def binary_search_sigma(clusters, perplexity, max_iter, tolerance):
    target_entropy = np.log(perplexity)
    distances = squareform(pdist(clusters))**2
    sigmas = []
    for i in range(clusters.shape[0]):
        current_row = distances[i, :]
        sigmamin = np.finfo(float).eps
        sigmamax = np.max(current_row)
        tries = 0
        Sigma = (sigmamin + sigmamax) / 2
        H, _ = compute_entropy(current_row, Sigma)
        while tries != max_iter and abs(H - target_entropy) > tolerance:
            if H > target_entropy:
                sigmamax = Sigma
            elif H < target_entropy:
                sigmamin = Sigma
            Sigma = (sigmamin + sigmamax) / 2
            H, _ = compute_entropy(current_row, Sigma)
            tries += 1
        sigmas.append(Sigma)
    return np.array(sigmas)


def compute_KLD_display(P, Q):
    KLD = np.sum(P * np.log(P / Q))
    return KLD

def gradientdescentvis(grid_res=0.75, x_start=-10, y_start=10, alpha=0.01, num_iterations=100, scale_factor=0.2,
                       sin_multiplier=1, sin_amplitude=5, x_bound=15, y_bound=15, plot_gradient_field=False,
                       gradient_field_scale_factor=0.05):

    x, y = np.meshgrid(np.arange(-x_bound, x_bound + grid_res, grid_res),
                       np.arange(-y_bound, y_bound + grid_res, grid_res))
    z = x**2 + y**2 + sin_amplitude*np.sin(sin_multiplier*x) + sin_amplitude*np.sin(sin_multiplier*y)

    x_current = x_start
    y_current = y_start
    z_current = x_current**2 + y_current**2 + sin_amplitude*np.sin(sin_multiplier*x_current) + sin_amplitude*np.sin(sin_multiplier*y_current)

    offset = 10

    x_trajectory, y_trajectory, z_trajectory = [x_current], [y_current], [z_current + offset]

    for i in range(num_iterations):
        x_previous = x_current
        y_previous = y_current
        z_previous = z_current

        dx = 2*x_current + sin_multiplier * sin_amplitude * np.cos(sin_multiplier * x_current)
        dy = 2*y_current + sin_multiplier * sin_amplitude * np.cos(sin_multiplier * y_current)

        x_current = x_current - alpha*dx
        y_current = y_current - alpha*dy
        z_current = x_current**2 + y_current**2 + sin_amplitude*np.sin(sin_multiplier*x_current) + sin_amplitude*np.sin(sin_multiplier*y_current)

        x_trajectory.append(x_current)
        y_trajectory.append(y_current)
        z_trajectory.append(z_current + offset)

    return x, y, z, x_trajectory, y_trajectory, z_trajectory

import numpy as np
import matplotlib.pyplot as plt

def plot_gradient_field(grid_res=0.75, sin_multiplier=1, sin_amplitude=5, x_bound=15, y_bound=15, gradient_field_scale_factor=0.05):

    x, y = np.meshgrid(np.arange(-x_bound, x_bound + grid_res, grid_res),
                       np.arange(-y_bound, y_bound + grid_res, grid_res))

    z = x**2 + y**2 + sin_amplitude*np.sin(sin_multiplier*x) + sin_amplitude*np.sin(sin_multiplier*y)

    # Calculate the gradient
    dx = 2*x + sin_multiplier * sin_amplitude * np.cos(sin_multiplier * x)
    dy = 2*y + sin_multiplier * sin_amplitude * np.cos(sin_multiplier * y)

    fig, ax = plt.subplots(figsize=(10, 8))
    ax.contourf(x, y, z, 50, cmap='viridis')  # Plotting the function
    ax.quiver(x, y, -gradient_field_scale_factor * dx, -gradient_field_scale_factor * dy, scale=25)  # Plotting the gradient vectors

    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    ax.set_title('Gradient Field')

    plt.show()

# Function to update the t-SNE visualization
def update_tsne(perplexity=30, exaggeration=3, alpha=190):
    tsne = TSNE(n_components=2, perplexity=perplexity, learning_rate=alpha, early_exaggeration=exaggeration, random_state=0)
    y_tsne = tsne.fit_transform(X)
    cmap = colormaps['viridis']

    plt.figure(figsize=(8, 6))
    plt.scatter(y_tsne[:, 0], y_tsne[:, 1], c=y, cmap=cmap, marker='o')
    plt.title(f"t-SNE Visualization (Perplexity={perplexity}, Exaggeration={exaggeration}, Alpha={alpha})")
    plt.colorbar(ticks=np.unique(y), label='Class')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.show()


Let's start by plotting two points each defined by two dimensions. Run this cell to see these points.

In [ ]:

# Define two points
p = [1, 2]
q = [4, 6]

# Create a new figure
plt.figure()

# Plot the points
plt.plot(p[0], p[1], 'ro', markersize=10, linewidth=2)  # 'ro' indicates red color, 'o' marker
plt.plot(q[0], q[1], 'bo', markersize=10, linewidth=2)  # 'bo' indicates blue color, 'o' marker

# Set x and y axis limits
plt.xlim([0, 8])
plt.ylim([0, 8])

# Draw the lines
plt.plot([p[0], q[0]], [p[1], p[1]], 'k-')  # Horizontal line
plt.plot([q[0], q[0]], [p[1], q[1]], 'k-')  # Vertical line

# Display the plot
plt.show()


Now let's look at what I mean by distance

In [ ]:
# Plot the line representing the distance between p and q
plt.figure()
plt.plot([p[0], q[0]], [p[1], q[1]], color='k', linewidth=1.5)

plt.plot(p[0], p[1], 'ro', markersize=10, linewidth=2)
plt.plot(q[0], q[1], 'bo', markersize=10, linewidth=2)

plt.plot(p[0], p[1], 'ro', markersize=10, linewidth=2)
plt.plot(q[0], q[1], 'bo', markersize=10, linewidth=2)
plt.plot([p[0], q[0]], [p[1], p[1]], 'k-')
plt.plot([q[0], q[0]], [p[1], q[1]], 'k-')

plt.xlim([0, 8])
plt.ylim([0, 8])

plt.show()


See, it's just Pythagoras!


An important thing to note, is that because we're doing rows of data (vectors), there is a slight change to the formula for distances, but not much and it is still the same idea.


it's given by:
  $\sqrt{\sum_{i = 1}^{n}({x_i - y_i)^2}}$

so if we have vectors:


$$\left\lbrack \begin{array}{ccc} 3 & 4 & 4 \end{array}\right\rbrack$$

$$\left\lbrack \begin{array}{ccc} 4 & 5 & 1 \end{array}\right\rbrack$$

$$\begin{array}{l} d(p,q)=\sqrt{(3-4)^2 +(4-5)^2 +(4-1)^2 }\\ =\sqrt{1+1+9}=\sqrt{11}. \end{array}$$


in code now

<pre>
def vector_distance(p, q):
    distance = 0
    for pi, qi in zip(p, q):
        distance += (pi - qi) ** 2
    return distance ** 0.5

# this is just for illustrative purposes.
# In practice you should use dist = numpy.linalg.norm(a-b)
</pre>

Boom, Now we are officically ready for the concept of a distance matrix!


A distance matrix is defined such that the ith row captures how far away it is from all the different datapoints.


So the ith row in the jth column tells you how far away data point i is from data point j.


In code:

<pre>
def dm(p):
    data_points, dim = p.shape
    distance_matrix = np.zeros((data_points, data_points))
    for i in range(data_points):
        for j in range(data_points):
            if i != j:
                sum_square_diff = sum([(p[i, k] - p[j, k])**2 for k in range(dim)])
                distance_matrix[i, j] = np.sqrt(sum_square_diff)
    return distance_matrix
</pre>

I should really note this, this code is for illustrative purposes, in practice, this can be succinctyly written with (using scipy):

<pre>
from scipy.spatial.distance import pdist, squareform

distance_matrix = squareform(pdist(p));
</pre>

The concept of a distance between two points can be concicely written with these bars:


$$\frac{||p-q||}{}$$

reads: the norm of the difference (distance) between vectors p and q.


Okay now that we have our distance matrix it's time to define our probability distribution matrix.


I like to think of a probability distribtuion as a function that takes in some parameters and spits out a prob


so instead of having a distance matrix we will have a probability distirbution matrix... It's the exact same prcocess just a different formula to plug your data points into.


Well, there is one slight difference worth talking about. There are two distributions we're going to use, a t and a normal or gaussian. We can focus on the gaussian for now. The gaussian takes in two parameters, a mean and a standard deviation. Once you give it those two parameters it becomes a well defined probability distribution.


Lets look at what happens when we change the mean of the Gaussian and hold the standard deviation constant.

In [ ]:
X = np.arange(-10, 10.01, 0.01)
Means = [0, -1, -2, 1, 1]

for mean_val in Means:
    gaussian = norm.pdf(X, mean_val, 1)
    plt.figure()
    plt.plot(X, gaussian)
    plt.title(f'A Gaussian with a mean of {mean_val}')
    plt.show()



Lets take a look at what happens when do the reverse; hold the mean constant and change the standard deviation.

In [ ]:
X = np.arange(-20, 20.01, 0.01)
STDs = [1, 2, 4, 6]
for std in STDs:
    gaussian = norm.pdf(X, 0, std)
    plt.figure()
    plt.plot(X, gaussian)
    plt.title(f'A Gaussian with a mean of 0 and a standard deviation of {std}')
    plt.show()




So there are two parameters we need for our high dimensional probability matrix P, we need the mean and the standard deviation. The mean is easy. For each data point we set it to be the mean of the gaussian.


The standard deviation is less easy, but we can get back to it.


Now lets look at how we define our high dimensional probability distribution matrix


$$p_{j|i} =\frac{\exp \left(-\frac{||x_i -x_j ||^2 }{2\sigma_i^2 }\right)}{\sum_{k\not= i} \exp \left(-\frac{||x_i -x_k ||^2 }{2\sigma_i^2 }\right)}$$

(Hint: that i below the sigma is meant to be there but dont worry about it too much for now)

What this is saying is we take each data point in our distance matrix, set it to be the mean and ask, how likely are each of the other datapoints?


Points that are far will get a low value and points that are close will get a high value.


Lets take a look at the numerator and see how it behaves with different distances.


In [ ]:
# Gaussian Curve
sigma = 1
denom1 = 2 * sigma**2

def numeratorfun(dist):
    return np.exp(-(dist**2) / denom1)

dists = np.arange(-5, 5.01, 0.01)
pji_notnormalize = [numeratorfun(dist) for dist in dists]

plt.figure()
plt.plot(dists, pji_notnormalize)
plt.title('Gaussian Distribution with Mean 0')
plt.xlabel('Distance from Mean')
plt.ylabel('Probability Density')
plt.grid(True)
plt.show()


Ah out pops the Gaussian. The denominator is there just to normalize the probability distribution. So each row is a conditional distribution which tells you how likely each other data point is as the neighbor of the row.

Now for a trick...

This is hard for me to motivate exactly. It helps to think in terms of what we are trying to do here. We want to capture the structures that exist in our data. In particular we're interested in the points that are more similar than other points, we're less interested in the global structure (why? Tradeoffs in reducing dimensions...) and more interested in the neighborhoods that each point is in.


One thing you could do, is define a standard deviation yourself... but how would you do that? You want to account for the fact that different data points should consider the same number of neighbors, but picking just one standard deviation for all of your data means that some points will get very few neighbors and others will get lots of neighbors. Tsne tries to capture the local structure of the data and this would be entirely removed if we used just one standard deviation.

Let's take a look at what I mean. First lets look at what happends with 5 random points and 1 uniform standard deviation.

In [ ]:
def single_std(sigma=0.5):
  # Scatter plot with single sigma
  np.random.seed(0)  # For reproducibility
  denseCluster = np.random.randn(5, 2)
  sparseCluster = np.random.randn(5, 2) * 3 + [5, 0]
  clusters = np.vstack([denseCluster, sparseCluster])

  plt.figure()
  plt.scatter(denseCluster[:, 0], denseCluster[:, 1], s=50, color='b', label='Dense Cluster')
  plt.scatter(sparseCluster[:, 0], sparseCluster[:, 1], s=50, color='r', label='Sparse Cluster')

  for point in clusters:
      circle = plt.Circle((point[0], point[1]), sigma, color='k', linestyle='-.', linewidth=1.5, fill=False)
      plt.gca().add_patch(circle)

  plt.title('Data Clusters with Single Sigma')
  plt.xlabel('X-axis')
  plt.ylabel('Y-axis')
  plt.legend()
  plt.grid(True)
  plt.show()
widgets.interactive(single_std, sigma=(0.5, 5, 0.01))



What ends up happening is that some points have no neighbors for a single σ.
Also, if a point p only has one neighbor q then they will be smooshed in the low dimensional representation even if they are really far away in the high dimensional data. The other point q might have p as a very unlikely neighbor, this will lead to significant problems in how we end up finding the low dimensional representation (the cost function).

Now let's take a look at what happens when we define a notion of nearest neighbors using a parameter called perplexity.

In [ ]:

def plot_clusters(perplexity=24):
    max_iter = 50
    tolerance = 1e-8
    sigmas = binary_search_sigma(clusters, perplexity, max_iter, tolerance)

    plt.figure(figsize=(10,6))
    plt.scatter(denseCluster[:, 0], denseCluster[:, 1], s=100, color='b', label='Dense Cluster')
    plt.scatter(sparseCluster[:, 0], sparseCluster[:, 1], s=100, color='r', label='Sparse Cluster')

    for i, point in enumerate(clusters):
        circle = plt.Circle((point[0], point[1]), sigmas[i], color='k', linestyle='-.', linewidth=1.5, fill=False)
        plt.gca().add_patch(circle)

    plt.title(f'Data Clusters with Perplexity-based Sigma (Perplexity = {perplexity})')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.legend()
    plt.grid(True)
    plt.xlim([-10, 10])
    plt.ylim([-10, 10])
    plt.show()

# Create an interactive widget
denseCluster = np.random.randn(5, 2)
sparseCluster = np.random.randn(5, 2) * 3 + [5, 0]
clusters = np.vstack([denseCluster, sparseCluster])
widgets.interactive(plot_clusters, perplexity=(2, 20, 1))




Perplexity is borrowed from information theory and is defined as:




$$H=-\sum_j p_{j|i} \log_2 p_{j|i}$$

$$\textrm{Perplexity}=2^H$$

H is the entropy, which measures the information of a discrete (it can do continuous too but it uses an integral instead of a sum) random variable. It does this by summing all the log probabibilities, the "suprisal" (its called  suprisal because improbale events have a high value) then scaling by the probability to determine how much it will contribute to the overall sum (more probable events will contribute more to the sum).
Entropy has the important property that when events are equally likely it is maximized, in otherwords, is the most uncertain it could be.


when you take $2^H$ you're effectively counting the total number of effective events.


lets do an example with a coin, C:

In [ ]:
Coin = [0.5,0.5];
entropy = 0;
for coin in Coin:
    entropy += coin * np.log2(coin);
entropy = -entropy
print(entropy)



So the entropy of a fair coin toss is one, in other words,  you get one bit of information when a coin is tossed.


Now lets look at what happens when take the perplexity:


In [ ]:
perplexity = 2**entropy
print(perplexity)


Ah... two possible outcomes.


So to bring it back, what tsne does is defines a user input perplexity, which roughly corresponds to the number of neighbors to consider for each point.


It does this by using a binary search to find a sigma for each datapoint such that when you compute the entropy of that distribution it matches the entropy given by the input perplexity. perplexity = log(entropy).


Here it is in code:

<pre>
import numpy as np
from scipy.spatial.distance import pdist, squareform

def binary_search_sigma(Data, perplexity, max_iter, tolerance):
    # max_iter = how many times to cut the search space in half
    # tolerance = how close do the target entropy and the distribution's entropy need to be.
    target_entropy = np.log(perplexity)
    distances = squareform(pdist(Data)) ** 2  # create distance matrix
    sigmas = np.zeros(Data.shape[0])  # initialize sigmas array
    for i in range(Data.shape[0]):  # for the number of data points (rows)
        current_row = distances[i, :]  # take the current data point.
        sigmamin = np.finfo(float).eps  # initialize the minimum possible sigma
        sigmamax = np.max(current_row)  # sigma can't be greater than the distance between current and furthest point
        tries = 0
        Sigma = (sigmamin + sigmamax) / 2  # compute the initial value for sigma
        entropy, _ = compute_entropy(current_row, Sigma)  # find the entropy of the current distribution
        while tries != max_iter and abs(entropy - target_entropy) > tolerance:
            # While we haven't tried the max number of times and the difference between entropy and target entropy is greater than a tolerance.
            if entropy > target_entropy:
                sigmamax = Sigma  # cut search space in half from the top
            elif entropy < target_entropy:
                sigmamin = Sigma  # cut search space in half from the bottom
            Sigma = (sigmamin + sigmamax) / 2  # compute new sigma
            entropy, _ = compute_entropy(current_row, Sigma)  # compute new entropy
            tries += 1
        sigmas[i] = Sigma
    return sigmas
</pre>

Run this cell to see what it looks like when we're looking for a point on a function: $F(X) = X^2$

In [ ]:
X = np.arange(-10, 10.1, 0.1)
Yfun = lambda X: X**2
Y = Yfun(X)
point_to_find = Y[10]
max_iter = 50

def plot_binary_search(step=0):
    plt.figure(figsize=(10,6))
    plt.plot(X, Y, label='Function')
    plt.plot(X[10], point_to_find, 'ro', linewidth=2, label='Point to Find')

    # Initializing binary search parameters
    min_x = np.min(X)
    max_x = np.max(X)
    tolerance = 1e-5
    guess = (min_x + max_x) / 2
    y_hat = Yfun(guess)
    tries = 0

    while tries <= step and tries < max_iter:
        if abs(y_hat - point_to_find) <= tolerance:
            break
        plt.plot(guess, y_hat, 'bx', linewidth=2, label='Guess' if tries == 0 else "")
        if y_hat > point_to_find:
            min_x = guess
        elif y_hat < point_to_find:
            max_x = guess
        guess = (min_x + max_x) / 2
        y_hat = Yfun(guess)
        tries += 1

    plt.legend()
    plt.show()

# Interactive widget
widgets.interactive(plot_binary_search, step=(0, max_iter-1))



Okay now we have a matrix, P(i|j), which represents the probability of point i picking point j as its neighbor. Unfortunately, these might not be symmetric, that is, the probability of picking point i given point j might not be the same as the probability of picking point j given point i. This is a problem because when we project into a lower dimension we want to have an interpetable relationship, and if we did it as so, with asymetry, when looking at the projection one would have to consider the direction of relationship.

(Big plus is simpler gradient)

The simple part here is actually symmetrizing which is given by:


$$P_{ij} =\frac{p_{j|i} +p_{i|j} }{2N}$$


The way to read this is: take the average of each point and divide by N (the number of data points you have). You divide by N to ensure it's a valid probability distribution over all the points, that is, it sums to 1.


Okay I'm going to recap now.


Step 1: Create a distance matrix (D) from our data (X)


Step 2: Compute pairwise affinitiy matrix (Pij) from X using a gaussian function with each data point as the center.


Step 2a: Find the standard deviation for each row, datapoint, using a binary search given the user input, perplexity.


Step 2b: Symmetrize our distribution.


Here is a sketch of the remainder of the steps:


&nbsp;&nbsp;&nbsp;&nbsp; Initialize low dimenisonal embedding (Y) from our data. All that means is that if our data originally had N row (datapoints) and M columns (dimensions, i.e, length, width, height, color, etc...) we want our "embedding" to have the same number of datapoints but with less columns, 2 or 3.


&nbsp;&nbsp;&nbsp;&nbsp; Perform steps 1 and 2 on Y such that we are left with a matrix (Qij) which represents the low dimensional pairwise affinities assuming a               students T distribution.


&nbsp;&nbsp;&nbsp;&nbsp; Compute a measure of difference between the two distributions (Kullback Leibler Divergence, KLD).


&nbsp;&nbsp;&nbsp;&nbsp; Minimize that difference between Qij and Pij by updating Y using gradient descent.


Initializing an embedding is easy, it looks like this:

<pre>
 Y = np.randn(N, 2);
</pre>

Then we compute a distance matrix:




<pre>
Distance = squareform(pdist(Y).^2
</pre>

Then we compute the pairwisee affinities Qij. The one difference here is that we use students T distribution with one DF, or Cauchy, to model our data instead of a Gaussian. We do this because of it's heavier tails. In other words, points that are at a moderate distance from each other in the High D space, will not have enough room in the low d space to spread out if we use a gaussian... that is, their distances will be way too high.


To show you what I mean:


In [ ]:
X = np.arange(-20, 20.01, 0.01)
DF = 1
sigma = 1
Mu = 0
T_distribution1 = t.pdf(X, DF)
Gaussian = norm.pdf(X, Mu, sigma)

plt.figure()
plt.plot(X, T_distribution1, label='T Distribution: DF = 1')
plt.plot(X, Gaussian, label='Gaussian: Mu = 0, Sigma = 1')
plt.title('Wider Tails of the T Distribution')
plt.legend(loc='upper right', fontsize='small')
plt.show()


A  T disttibution takes in 1 parameter, the degrees of freedom, in TSNE we just use a value of 1.


That gives us this way of computing our pairwise affinities in the low dimensional space:


$$q_{ij} =\frac{\exp (-||y_i -y_j ||^2 )}{\sum_{k\not= l} \exp (-||y_k -y_l ||^2 )}$$

In code:

<pre>
def compute_lowD_P_affin(Y):
    low_D = squareform(pdist(Y))**2  # distance matrix
    num = (1 + low_D)**-1  # numerator
    np.fill_diagonal(num, 0)  # set diagonals to zero
    qij = num / (np.sum(num) + np.finfo(float).eps)  # divide by the normalizing value.
    return qij
</pre>

So now we need a way of measuring how different these two distributions are. This is where Kullback-Leibler Divergence comes into play, woohoo!


DKL Explainer


DKL is a way of asking, given my data (P(i)), how suprised should I be if I found this data (Q(i)).


It's given by:

$$D_{KL} (P||Q)=\sum_i P(i)\log \frac{P(i)}{Q(i)}\textrm{}$$

In code:

<pre>
def compute_KLD(P, Q):
    KLD = np.sum(P * np.log(P / Q))
    return KLD
</pre>

for Tsne, it looks like


$$\begin{array}{l} C=\sum_{i\not= j} p_{ij} \log \frac{p_{ij} }{q_{ij} }\\  \end{array}$$

What this is saying is compute, for all datapoints the KLD between the low and high dimensional space, except for the diagonals.



Run this cell and play with the slider to see how when two distributions are far away KLD is high and when they are close it is low.

In [ ]:
# Define the x-axis
x = np.linspace(-10, 10, 1000)

# Parameters for P: mean = 2, standard deviation = 1
mu_P = 2
sigma_P = 1
sigma_Q = 1

# Define the Gaussian distribution P
P = norm.pdf(x, mu_P, sigma_P)

def plot_distributions(mu_Q=2.5, sigma_Q=1):
    """Plot distributions P and Q with varying mu_Q."""
    # Define the Gaussian distribution Q
    Q = norm.pdf(x, mu_Q, sigma_Q)

    # Compute KLD
    KLD = compute_KLD_display(P, Q)

    # Plotting
    plt.figure(figsize=(10,6))
    plt.fill_between(x, P, color='r', alpha=0.3)
    plt.fill_between(x, Q, color='b', alpha=0.3)
    plt.plot(x, P, 'r-', linewidth=2)
    plt.plot(x, Q, 'b-', linewidth=2)
    plt.xlabel('x')
    plt.ylabel('Probability Density')
    titleStr = f'Distributions P and Q with KLD = {KLD:.4f}'
    plt.title(titleStr)
    plt.legend(['P (True distribution)', 'Q (Approximation)'])
    plt.grid(True)
    plt.show()

# Interactive widget with slider for mu_Q
widgets.interactive(plot_distributions, mu_Q=(-5, 5, 0.1))




Okay, now that we have a value we can minimize: KLD, how do we do that?


It's time for a detour, woohoo!!!


Say we have a function:


$$F(X,Y)=X^2 +Y^2$$

And we want to find the minimum of that function. Well we can go back to calculus and take the concept of the gradient. The gradient of a function is defined as a vector of all the first order partial derivatives of a function. What the gradient tells you is the direction of steepest ascent and how much to care about each variable on your climb.


The gradient of a function for multiple variables:


$$F(X):X=(x_1 ,x_2 ,\ldots,x_n )$$

$$\nabla F(X)=\left(\frac{\partial F}{\partial x_1 },\frac{\partial F}{\partial x_2 },\ldots,\frac{\partial F}{\partial x_n }\right)$$

Thus the gradient of the function:


$$F(X)=x^2 +y^2$$

Is:


$$\frac{\partial F}{\partial x}=2x$$

$$\frac{\partial F}{\partial y}=2y$$

$$\nabla F(X)=(2x,2y)$$

The update rule is given by:


$$X_{new} =X_{old} -\alpha \nabla F(X_{old} )$$

Where $\alpha$ is your learning rate and $\nabla F(X_{old} )$ is your gradient. This is how big you want your step size to be. It actually looks quite simple in code:

<pre>
def F(X, Y):
    return X**2 + Y**2

Alpha = 0.1
X_current = -5
Y_current = -5
Z_current = F(X_current, Y_current)
num_iterations = 100

for i in range(num_iterations):
    X_previous = X_current
    Y_previous = Y_current
    Z_previous = Z_current

    dx = 2 * X_current  # gradient element 1
    dy = 2 * Y_current  # gradient element 2

    X_current = X_current - Alpha * dx
    Y_current = Y_current - Alpha * dy
    Z_current = F(X_current, Y_current)


</pre>




To see what it looks like run this cell. You can adjust the parameter alpha and run the cell again to see how the path changes.

In [ ]:
alpha = 0.01 #learningrate: play with this to see how the path changes
x, y, z, x_trajectory, y_trajectory, z_trajectory = gradientdescentvis(alpha=alpha)

def plot_gradient_descent(step=0):
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(x, y, z, cmap=cm.coolwarm, edgecolor='none', alpha=0.7)
    ax.view_init(elev=51, azim=42)

    ax.plot(x_trajectory[:step+1], y_trajectory[:step+1], z_trajectory[:step+1], 'r-', linewidth=2)
    ax.quiver(x_trajectory[step], y_trajectory[step], z_trajectory[step],
              x_trajectory[step] - x_trajectory[step-1],
              y_trajectory[step] - y_trajectory[step-1],
              z_trajectory[step] - z_trajectory[step-1],
              color='b', linewidth=2, pivot='tail')

    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    ax.set_zlabel('Cost')
    ax.set_title('Gradient Descent')
    plt.show()

widgets.interactive(plot_gradient_descent, step=(0, len(x_trajectory)-1))

Run this cell to display the Gradient Field (gradient evaluated at lots of points on a function) on the function
$$F(x,y) = x^2 + y^2 + \alpha \sin(\beta x) + \alpha \sin(\beta y)$$

where $\alpha$ represents the amplitude of the sin wave and $\beta$ represents the sin multiplier.

$$\frac{\partial z}{\partial x} = 2x + \beta \alpha \cos(\beta x)$$
$$\frac{\partial z}{\partial y} = 2y + \beta \alpha \cos(\beta y)$$

$$\nabla F(x,y) = \left[ \frac{\partial z}{\partial x}, \frac{\partial z}{\partial y} \right] = \left[ 2x + \beta \alpha \cos(\beta x), 2y + \beta \alpha \cos(\beta y) \right]$$





In [ ]:
plot_gradient_field()


Okay so now we know what gradient descent is. Let's put it all together. In practice the gradient of the cost function for tsne is given by:


$$\frac{\partial C}{\partial y_i }=4\sum_j (p_{ij} -q_{ij} )(y_i -y_j )(1+||y_i -y_j ||^2 )^{-1}$$


This can be interpeted as consisting of an attractive and a repulsive force. If Pij is higher than Qij, than the force exerted on that point, i, will be attractive, because the affinities are bigger in high than low. The repulsive force is exterted in the opposite way.


The attractive term:


$$(p_{ij} )(y_i -y_j )$$

The repulsive term:


$$(q_{ij} )(y_i -y_j )$$

These are subtracted to get a total force exerted.


The final term in the equation comes from the definition of the T distribution and determines how much to care about each point.


As the distance $||y_i -y_j ||$ between two points increases, the term $(1+||y_i -y_j ||^2 )^{-1}$ approaches zero. This ensures that algorithim tracks local structure in the high dimensional space.


Finally! Let's see it in action and how it behaves with different parameters.


Remember: There are four columns: Sepal length, Sepal width, Petal length, Petal width. One flower (observation) per row.

In [ ]:
# Load the Fisher Iris dataset
iris = load_iris()
X = iris.data
y = iris.target


# Create sliders
perplexity_slider = widgets.FloatSlider(value=30, min=1, max=len(y)-1, step=1, description='Perplexity:')
exaggeration_slider = widgets.FloatSlider(value=3, min=1, max=1000, step=0.5, description='Exaggeration:')
alpha_slider = widgets.FloatSlider(value=190, min=1, max=1000, step=10, description='Alpha:')

# Create interactive widget
widgets.interactive(update_tsne, perplexity=perplexity_slider, exaggeration=exaggeration_slider, alpha=alpha_slider)
